In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import requests
import io
import time
import pygsheets
from collections import defaultdict
import json

In [ ]:
headers = {
    'Authorization': 'Bearer token'
}


In [ ]:
GSHEET_URL = 'custom_params_url'

In [ ]:
def get_params(gsheets_url: str, headers: dict = None):

    # get custom params from google sheet table
    gc = pygsheets.authorize(service_account_env_var="SERVICE_ACCOUNT_KEY")
    spreadsheet = gc.open_by_url(gsheets_url)
    worksheet = spreadsheet.worksheet_by_title("active_clients")
    df_sheet = worksheet.get_as_df()
    df_sheet = df_sheet.applymap(lambda x: str(x).replace(' ', ''))

    custom_params = defaultdict(dict)
    for app, reattributed, attribution_type, attribution_source in zip(df_sheet['app_token'], df_sheet['reattributed'], df_sheet['attribution_type'], df_sheet['attribution_source']):
        custom_params[app]['reattributed'] = reattributed
        custom_params[app]['attribution_type'] = attribution_type
        custom_params[app]['attribution_source'] = attribution_source

    # get account ids
    response = requests.get("https://www.adjust.com/api/accounts", headers=headers)
    accounts = response.json()["accounts"]

    # get events
    df_events = pd.DataFrame()
    for i in accounts:
        headers["x-account-id"] = str(i["id"])

        response = requests.get(
            "https://dash.adjust.com/control-center/reports-service/events",
            headers=headers,
        )

        df = pd.DataFrame(response.json())
        try:
            df = df[["app_token", "name"]]
            df.columns = ["app_token", "event_name"]
        except Exception as error:
            continue
        df["account_name"] = i["name"]
        df["x-account-id"] = i["id"]

        df_events = pd.concat([df_events, df], axis=0)

    df_events.dropna(subset=['x-account-id'], inplace=True)

    df_events = df_events.explode("app_token").reset_index(drop=True)
    df_events['event_name'] = df_events['event_name'].str.lower() + '_events,'
    df_events = df_events[~df_events['event_name'].str.contains(' ')]
    df_events = df_events.groupby(['app_token', 'x-account-id', 'account_name']).sum().reset_index()
    df_events['event_name'] = df_events['event_name'].str[:-1]
    # df_events['event_name'] = df_events.groupby(['app_token', 'x-account-id'])['event_name'].transform(lambda x: '_events,'.join(x))

    event_dict = defaultdict(dict)
    for app, x_account_id, account_name, event_name in zip(df_events['app_token'], df_events['x-account-id'], df_events['account_name'], df_events['event_name']):
        event_dict[app]['x-account-id'] = x_account_id
        event_dict[app]['account_name'] = account_name
        event_dict[app]['event_name'] = event_name


    # merge custom params with event list
    all_params = custom_params

    for app, params in event_dict.items():
        if app in all_params.keys():
            for param, value in params.items():
                all_params[app][param] = value
        else:
            pass

    return all_params


In [ ]:
def adjust_overview(headers: dict, parametrs: dict):
    df_adj = pd.DataFrame()
    for app, param in parametrs.items():

        print(app)

        headers["x-account-id"] = str(param["x-account-id"])

        for reattributed in param["reattributed"].split(","):
            for attribution_type in param["attribution_type"].split(","):
                for attribution_source in param["attribution_source"].split(","):

                    params = {
                        "app_token__in": app,
                        "metrics": "installs," + param["event_name"],
                        "dimensions": "day,app,app_token,os_name,partner,partner_name,network,campaign,adgroup_network,adgroup,creative",
                        "date_period": "2024-04-01:2024-04-02",
                        "attribution_type": attribution_type,
                        "attribution_source": attribution_source,
                        "utc_offset": "+03:00",
                    }

                    response = requests.get(
                        "https://dash.adjust.com/control-center/reports-service/csv_report?",
                        headers=headers,
                        params=params,
                    )

                    if response.status_code == 200:
                        df = pd.read_csv(io.StringIO(response.text), low_memory=False)
                        df["x-account-id"] = param["x-account-id"]
                        df["reattributed"] = reattributed
                        df["attribution_type"] = attribution_type
                        df["attribution_source"] = attribution_source

                        dimensions = [
                            "day",
                            "x-account-id",
                            "app",
                            "app_token",
                            "os_name",
                            "partner",
                            "partner_name",
                            "network",
                            "campaign",
                            "adgroup_network",
                            "adgroup",
                            "creative",
                            "reattributed",
                            "attribution_type",
                            "attribution_source",
                        ]
                        df = df[dimensions + [i for i in df.columns if i not in dimensions]]
                        df_adj = pd.concat([df_adj, df], axis=0)
                    else:
                        print(app, " - ", response.text)

    return df_adj


In [ ]:
p = get_params(GSHEET_URL, headers=headers)
adj = adjust_overview(headers=headers, parametrs=p)
adj.head()